# Bitcoin price forecasting with PySpark
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



### Introduction

The cryptocurrency Bitcoin has attracted the attention of many people in recent years. However, it's
price fluctuation can be extremely unpredictable, which makes it difficult to predict when the right
time to buy or sell this digital currency will be. In this context, forecasting Bitcoin prices can be a
competitive advantage for investors and traders, as it could allow them to make informed decisions
on the right time to enter or exit the market. In this project, I will analyze some machine learning
techniques to understand, through the processing of historical data, how accurately the price of Bitcoin
can be predicted and whether this can provide added value to cryptocurrency investors and traders.
### Dataset
I chose to use the following dataset from Kaggle Bitcoin Historical Dataset, more specifically those
containing minute-by-minute updates of the Bitcoin price from 2017 to 2021 (period for which there
were moments of high volatility but also a lot of price lateralisation). The columns (features) contained
in it, in addition to the timestamp of each transaction, are the opening, closing, highest and lowest
price and the corresponding trading volume in Bitcoin and Dollars.
### Methods (TODO: da scegliere per bene)
The methods I will test will be Linear Regression (simple and multiple) and Random Forest. Further
comparisons with other classification models are planned in the course of development. Moreover, I
would also like to try to understand what the differences are between these methods and the imple-
mentation of a state-of-the-art neural network such as Long-Short Term Memory.
### Evaluation framework (TODO: vedi quali usare in base ai paper/esempi e ai modelli utilizzati)
As evaluation framework I will use R-square (R²), Mean Square Error (MSE) and Mean Absolute
Error (MAE) to get a complete picture of the performance of the various models.

# Global Constants


In [1]:
# TODO: da sistemare

JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/Computer_Science/BDC/project/datasets/output"
GDRIVE_DATASET_TEMP_DIR = GDRIVE_DIR + "/MyDrive/Computer_Science/BDC/project/datasets/temp"

# GDRIVE_DATASET_NAME_TRAIN = "data_cleaned_train"
# GDRIVE_DATASET_NAME_TEST = "data_cleaned_test"
# GDRIVE_DATASET_NAME_TRAIN = "1M_cleaned_train"
# GDRIVE_DATASET_NAME_TEST = "1M_cleaned_test"
GDRIVE_DATASET_NAME_TRAIN = "800K_cleaned_train"
GDRIVE_DATASET_NAME_TEST = "800K_cleaned_test"

GDRIVE_DATASET_NAME_EXT_TRAIN  = "/" + GDRIVE_DATASET_NAME_TRAIN + ".csv"
GDRIVE_DATASET_NAME_EXT_TEST = "/" + GDRIVE_DATASET_NAME_TEST + ".csv"

GDRIVE_DATASET_TRAIN = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TRAIN
GDRIVE_DATASET_TEST = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TEST

SLOW_OPERATION = False

#  Import useful Python packages

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

# **Spark + Google Colab Setup**

## Install PySpark and related dependencies





In [3]:
!pip install pyspark
# Alternatively, if you want to install a specific version of pyspark:
#!pip install pyspark==3.2.1
!pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.sql import functions as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=97fa8a4bfc33cf8b119d3550723ef27ffeb4b45573490308bb8e064f8ab2d360
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

##  Create Spark context

In [4]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

##  Link Colab to our Google Drive

In [5]:
# Point Colaboratory to our Google Drive

from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


##  Check everything is ok

In [6]:
spark

In [7]:
sc._conf.getAll()

[('spark.driver.memory', '45G'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.name', 'BitcoinPriceForecasting'),
 ('spark.app.submitTime', '1686412896502'),
 ('spark.driver.host', 'c893f6a92c3a'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-open

#  XGBoost

https://xgboost.readthedocs.io/en/stable/parameter.html#global-configuration

In [8]:
# load dataset into pyspark dataframe objects
train_df = spark.read.load(GDRIVE_DATASET_TRAIN, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                    )

test_df = spark.read.load(GDRIVE_DATASET_TEST, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                    )

In [9]:
if SLOW_OPERATION:
  print("The shape of the train dataset is {:d} rows by {:d} columns".format(train_df.count(), len(train_df.columns)))
  train_df.show(5)
  print("The shape of the test dataset is {:d} rows by {:d} columns".format(test_df.count(), len(test_df.columns)))
  test_df.show(5)

In [14]:
def xgboost(train, test):
  from pyspark.ml.regression import RandomForestRegressor
  from pyspark.ml.feature import StringIndexer, VectorAssembler

  # TODO: da risolvere in qualche modo (devo ricalcolarmi il vettore "features" perché quando importo i dataset viene visto come una stringa)
  train = train.drop("features")
  test = test.drop("features")

  assembler = VectorAssembler(
      inputCols=["close"], 
      outputCol="features"
  )

  train = assembler.transform(train)
  test = assembler.transform(test)

  !pip install xgboost
  from xgboost.spark import SparkXGBRegressor

  # Creazione dell'XGBoostRegressor
  xgboost = SparkXGBRegressor(
      features_col="features",  # Colonna vettoriale delle features
      label_col="labelIndex",  # Colonna delle etichette di output
  )
  
  # Addestra il modello: Utilizza il metodo fit() per addestrare il modello sulla tua dataset di addestramento.
  model = xgboost.fit(train)

  # Effettua le previsioni: Utilizza il modello addestrato per fare previsioni sul tuo dataset di test o su nuovi dati.
  predictions = model.transform(test)

  return predictions

In [15]:
predictions = xgboost(train_df, test_df)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:782: UserWarning:

Loading a native XGBoost model with Scikit-Learn interface.



In [17]:
def evaluation(pred):
  from pyspark.ml.evaluation import RegressionEvaluator

  evaluator = RegressionEvaluator(
      labelCol="labelIndex",  # Colonna delle etichette di output
      predictionCol="prediction"  # Colonna delle previsioni
  )

  mse = evaluator.evaluate(pred, {evaluator.metricName: "mse"})
  rmse = evaluator.evaluate(pred, {evaluator.metricName: "rmse"})
  r2 = evaluator.evaluate(pred, {evaluator.metricName: "r2"})
  mae = evaluator.evaluate(pred, {evaluator.metricName: "mae"})

  from pyspark.sql.functions import abs, col
  from pyspark.sql import functions as F
  from pyspark.ml.evaluation import RegressionEvaluator

  # Calcola il MAPE
  mape = pred.withColumn("error", abs(col("labelIndex") - col("prediction")) / col("labelIndex")) \
          .selectExpr("avg(error) * 100 as mape") \
          .collect()[0]["mape"]
          
  # adj_r2 
  n = pred.count()  # Numero di osservazioni
  p = 1  # Numero di predittori nel modello
  adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

  print("MSE = %s" % (mse)) # deve essere un valore non negativo, dove un valore di 0 indica una perfetta corrispondenza tra i valori predetti e quelli di riferimento
  print("RMSE = %s" % (rmse)) # dovresti considerare il valore di RMSE in relazione al range dei valori target nel tuo problema specifico
  print("R2 = %s" % (r2)) # piú é vicino ad 1 meglio é
  print("MAE = %s" % (mae)) # può essere utile confrontare il valore di MAE con quello di altri modelli o con il range dei valori target per valutare la sua precisione
  print("MAPE = %s" % (mape)) # di solito viene utilizzato come misura relativa per confrontare la precisione di modelli diversi
  print("ADJ R2 = %s" % (adj_r2)) # piú é vicino ad 1 meglio é

In [18]:
evaluation(predictions)

MSE = 39383640744.09316
RMSE = 198453.11976407215
R2 = -16.023060161006043
MAE = 188839.66889321423
MAPE = 24.808036443946772
ADJ R2 = -16.02316232858694
